# Programowanie obiektowe

W paradygmacie programowania obiektowego, klasy stanowią najważniejszy budulec programów. Niemniej jednak w języku Python istnieje wiele typów i technik, które pozwalają ograniczyć ich użycie. W językach programowania takich jak Java czy C#, klasy używane są zarówno jako kontenery do danych jak i typowe obiekty, które są wyspecjalizowane do konkretnych zadań. W języku Python jako kontenerów można użyć typu `namedtuple` lub `dataclass`. Nową techniką unikalną dla języka Python, która nie występuje w innych językach programowania jest tzw. _mixin_, czyli wstrzykiwanie pojedynczych funkcjonalności i użycie dziedziczenia jako kompozycji. Przykładem może być tutaj logowanie błędów. Jest to dość oczywista funkcjonalność jednak może być realizowana na wiele sposób np. logowanie na konsole, do pliku, usługi ElasticSearch czy S3 (w postaci plików).

## Definiowanie klas

Do definicji klasy należy użyć słowa kluczowego `class`, a następnie przekazać jej identyfikator typu. Opcjonalne dziedziczenie przekazuje się za pomocą nawiasu, zaraz za identyfikatorem. Poniżej znajduje się przykładowa deklaracja klasy.

In [2]:
class Book:
    isbn_field: str = 'isbn'

    def __init__(self, isbn: str):
        self.isbn = isbn

    def validate(self) -> bool:
        return len(self.isbn) > 0

book = Book('2211-1212')
book.validate()

True

Dość istotny jest tutaj brak modyfikatorów dostępu znanych z innych języków programowania. Domyślnie w języku Python wszystko jest dostępne jako publiczne. Jedynym ograniczeniem jest stosowanie podkreślenia przed nazwą funkcji lub zmiennej, mający symulować pola prywatne. Niemniej jednak jest to bardziej notacja niż rzeczywiste ograniczenia na poziomie kompilacji. Pole `isbn_field` jest statyczne i odwołać się do niego można bez inicjalizacji klasy (`Book.isbn_field`). Konstruktor `__init__` tworzy zmienne dostępne w trakcie działania programu i użycia klasy. Nowością w stosunku do innych języków programowania jest słowo `self` odwołujące się do bieżącej instancji klasy. Funkcja `validate` może być jedynie wywołana po utworzeniu instancji klasy. W tworzeniu instancji klasy nie stosuje się słowa kluczowego `new`.

## Dziedziczenie

W języku Python dziedziczenie jest wielokrotne tzn. klasa może dziedziczyć po wielu klasach na raz. Dodatkowo priorytet  dziedziczenia jest od prawej do lewej, co ma szczególne znaczenie w momencie, gdy więcej niż jedna klasa zawiera tą samą nazwę funkcji. Kolejny przykład przedstawia taki właśnie przypadek.

In [1]:
class A:
    def test(self):
        print('Class A')

class B:
    def test(self):
        print('Class B')

class C (A, B):
    def print(self):
        self.test()


c = C()
c.print()

Class A


W przypadku, gdyby na liście dziedziczenia, klasa `B` była przed `A`, wtedy funkcja klasy `B` została by użyta przy wywołaniu. Technika ta ma za zadanie rozwiązanie problemu diamentu (_The diamond problem_ w C++), który polega na tym, że kompilator nie jest w stanie określić, której funkcji klasy pochodnej ma użyć w trakcie wywołania. Dzieje się tak, gdy obie funkcje implementują abstrakcyjną klasę, a pewna klasa szczegółowa dziedziczy z obu z nich. Wywołanie konstruktora klasy bazowej następuje w trakcie wywołania funkcji `super().__init__()` lub poprzez precyzyjniejsze wywołanie `BaseClass.__init__()`, gdzie `BaseClass` jest typem bazowym.

In [5]:
class A:
    def __init__(self):
        print('Class A')

class B:
    def __init__(self):
        print('Class B')

class C (A, B):
    def __init__(self):
        print('Class C')
        print('Calling super')
        super().__init__()
        print('Calling directly A.__init__')
        A.__init__(self)
        print('Calling directly B.__init__')
        B.__init__(self)

c = C()

Class C
Calling super
Class A
Calling directly A.__init__
Class A
Calling directly B.__init__
Class B


Powyższy przykład prezentował sposób wywołania konstruktorów bazowych, jednak tą samą technikę można wykorzystać przy wywoływaniu funkcji.

## Anotacja `property`

Z definicji klasa zawiera zmienne i funkcje, które określają jej cechy. Jak dotąd przedstawiony był sposób tworzenia zmiennych w konstruktorze i definiowania funkcji. Zmienne wchodzące w skład klasy można definiować również jako właściwości. Dostęp do nich wygląda podobnie jak dostęp do pól.

In [ ]:
class AuthorBook:
    def __init__(self, author: str, title: str):
        self._author = author
        self._title = title

    @property
    def title(self):
        return self._title.title()

    @title.setter
    def title(self, title: str):
        self._title = title
        # dodatkowa logika może zostać przekazana tutaj

book = AuthorBook('Andrzej Sapkowski', 'Wiedźmin')
book.title

Stosowanie właściwości ma na celu umożliwienie programiście dodania kodu zawierającego logikę związaną z obsługą zmiany pola (np. przeliczenie powiązanych pól). Bardziej zaawansowanym przykładem stosowania pól jest użycie funkcji specjalnych `__get__` i `__set__`, które umożliwiają tworzenie własnych właściwości bez konieczności stosowania powyższej nomenklatury (kosztem czytelności i prostoty kodu).

## Anotacje `staticmethod` i `classmethod`

Funkcje statyczne są dość powszechnie stosowane w językach programowania obiektowego. Są to głównie funkcje, niezwiązane z konkretną instancją klasy, ale pod względem funkcjonalnym musi być z nią związana. W języku Python, aby funkcja była statyczna należy dodać anotację `staticmethod`. W argumentach nie może zawierać wskazania `self`, gdyż nie jest w żaden sposób związana z konkretną instancją klasy.

In [ ]:
class A:
    @staticmethod
    def static_test():
        pass

A.static_test()

Druga anotacja ma zastosowanie do inicjalizacji klasy w sposób niestandardowy. Przykładem może byc manualna serializacja i deserializacja  np. ze słownika.

In [21]:
from os import listdir
from os.path import join


default_data_dir = './data'

class PathInput:
    def __init__(self, path):
        self.fullpath = path

    @classmethod
    def generate_inputs(cls, data_dir):
        data_dir = data_dir or default_data_dir

        for name in listdir(data_dir):
             yield cls(join(data_dir, name))

for path_input in PathInput.generate_inputs('./'):
    pass

Funkcja `generate_inputs` nie zawiera podobnie jak funkcja statyczna zmiennej `self` wskazującej wartości zmiennych konkretnej instancji klasy. Podstawą różnicą jest argument `cls`, który pozwala tworzyć nową instancję klasy i ulega dziedziczeniu (w nowym typie po dziedziczeniu będzie wskazywał na nowy typ).

## Definiowanie klas abstrakcyjnych

Język Python nie zawiera interfejsów, a jedynie typy abstrakcyjne. Poniżej znajduje się przykład tworzenia klasy abstrakcyjnej.

In [ ]:
import abc

class Shape(metaclass=abc.ABCMeta):
    @abc.abstractmethod
    def area(self):
        pass

class Rectangle(Shape):
   def __init__(self, x,y):
      self.x=x
      self.y=y
   def area(self):
      return self.x*self.y

W momencie, gdy klasa dziedziczy po typie abstrakcyjnym, lecz nie implementuje jednej z abstrakcyjnych funkcji, środowisko uruchomieniowe wywoła wyjątek `TypeError`.

## Mixin

Dziedziczenie wielokrotne w języku Python umożliwia tworzenie bardzo wyspecjalizowanych klas, które można wstrzykiwać w hierarchię dziedziczenia, co często nazywane jest kompozycją funkcji. Należy również pamiętać, że nie jest to technika wspierana w samym języku Python, a sposób projektowania klas.

In [ ]:
class SerializerMixin:
    def Write(self, dict):
        pass

class SerializableRectangle(SerializerMixin, Rectangle):
    pass

Stosowanie powyższej techniki pozwala znacząco uprościć kod i jest zgodna z zasadą _DRY_ (Don't Repeat Yourself).

## Kontenery na dane

Anotacja `dataclass` umożliwia tworzenie prostych klas, których głównym zadaniem jest przechowywanie danych. Implementuje ona podstawowe operacje na danych jak porównywanie czy wyświetlanie informacji (implementacja funkcji `__repr__`).

In [14]:
from dataclasses import dataclass, field, asdict
from typing import List

@dataclass
class Employee:
    name: str
    surname: str = field(default='Not assigned')
    band: List[int] = field(default_factory=list) # wartość inicjalna []


emp = Employee(name='Lukasz', surname='Strak')
print(emp.name)
print(asdict(emp))

Lukasz
{'name': 'Lukasz', 'surname': 'Strak', 'band': []}


Użycie modułu `dataclass` pozwala zaoszczędzić wiele czasu na ręcznym tworzeniu klas.

## Enums

Enumeracje w języku Python są realizowane jako klasa dziedzicząca po `Enum`, a same wyliczenia przechowywane są w statycznych polach, tak jak pokazano w poniższym przykładzie.

In [16]:
from enum import Enum

class Color(Enum):
    RED = 0
    GREEN = 1
    Yellow = 2


def print_color(color: Color):
    print(color.name)
    print(color == Color.RED)


print_color(Color.RED)

RED
True


## Informacje o obiekcie w czasie wykonania

Biblioteka standardowa zawiera wiele przydatnych funkcji, które umożliwiają uzyskanie informacji o instancji obiektu w czasie dziania programu.

In [20]:
class A:
    pass

class B(A):
    def test(self):
        pass

# wyświetla typ danych
print(type(B))

# dir zwraca wszystkie składowe klasy
print(dir(B()))

# mro pozwala wyświetlić całą hierarchię dziedziczenia
print(type(B).mro(B))

# hasattr to funkcja wbudowana w język Python sprawdzająca czy dany typ zawiera funkcję lub pole dane łańcuchem znaków
print(hasattr(B,'test'))

<class 'type'>
['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'test']
[<class '__main__.B'>, <class '__main__.A'>, <class 'object'>]
True


## Zadania do wykonania

### Zadanie 1. Zaprojektuj klasę dla $kd$-drzewa.

### Zadanie 2. Zaimplementuj wzorzec projektowy łańcuch odpowiedzialności na przykładzie obsługi żądania _http_ (symulacja), w którym przed faktycznym kodem obsługi błędu ma zostać sprawdzone czy użytkownik może wysłać danego typu żądanie, czy żądanie nie dotyczy pliku, czy liczba żądań na minutę nie jest przekroczona, czy liczba żądań na minutę nie jest przekroczona dla zalogowanego użytkownika, czy przesłany formularz nie jest próbą `sql incjection`.

### Zadanie 3. Za pomocą dowolnego wzorca projektowego zaimplementuj mechanizm sprawdzający poprawność wyrażenia postaci:
* a + b = c (poprawne),
* (a + b = c (niepoprawne),
* a + = c (niepoprawne).

In [ ]:
from __future__ import annotations
from collections import namedtuple
from operator import itemgetter
from typing import Any, Optional
from abc import ABC, abstractmethod
from pprint import pformat

# ZADANIE 1
class Node(namedtuple("Node", "loc left_child right_child")):
    def __repr__(self):
        return pformat(tuple(self))
    

def kd_tree(kdtree_list, depth: int = 0):
    if not kdtree_list:
        return None

    k = len(kdtree_list[0])
    ax = depth % k

    kdtree_list.sort(key=itemgetter(ax))
    median = len(kdtree_list) // 2

    return Node(
        loc=kdtree_list[median],
        left_child=kd_tree(kdtree_list[:median], depth + 1),
        right_child=kd_tree(kdtree_list[median + 1 :], depth + 1),
    )


def main():
    point_list = [(1, 3), (6, 2), (8, 5), (3, 1), (5, 8), (2, 4)]
    tree = kd_tree(point_list)
    print(tree)

# ZADANIE 2
class Context():

    def __init__(self, strategy: Strategy) -> None:
        self._strategy = strategy

    @property
    def strategy(self) -> Strategy:
        return self._strategy

    @strategy.setter
    def strategy(self, strategy: Strategy) -> None:
        self._strategy = strategy

    def do_some_business_logic(self) -> None:
        # ...

        val = "(a + b) = c"
        print(f"Wyrażenie arytmetyczne: {val}")
        result = self._strategy.do_algorithm(val)
        print(result)

        # ...


class Strategy(ABC):

    @abstractmethod
    def do_algorithm(self, string):
        pass


class ConcreteStrategyA(Strategy):
    def do_algorithm(self, string):
        valid: string = "Poprawne"
        invalid: string = "Niepoprawne"
        is_good: bool = False
        is_not_mistaken: bool = False
        counter_left = 0
        counter_right = 0
        arr = string.split(" ")

        string_2 = string.replace('(','')
        string_for_checking = string_2.replace(')','')
        arr_for_checking = string_for_checking.split(" ")

        for i in range(0, len(arr)):
            if arr[i] == "(":
                counter_left = counter_left + 1
            elif arr[i] == ")":
                counter_right = counter_right + 1

        for i in range(1, len(arr)):
            if not arr_for_checking[i].isalpha() and not arr_for_checking[i - 1].isalpha():
                is_not_mistaken = False
                break
            else:
                is_not_mistaken = True

        if counter_left == counter_left:
            is_good = True

        if is_good and is_not_mistaken:
            return valid
        else:
            return invalid


# ZADANIE 3
class Req:
  def __init__(self, name, authorized, is_file, request_amount, is_sql_injection):
    self.name = name
    self.authorized = authorized
    self.is_file = is_file
    self.request_amount = request_amount
    self.is_sql_injection = is_sql_injection


class Handler(ABC):

    @abstractmethod
    def set_next(self, handler: Handler) -> Handler:
        pass

    @abstractmethod
    def handle(self, request) -> Optional[str]:
        pass


class AbstractHandler(Handler):
    _next_handler: Handler = None

    def set_next(self, handler: Handler) -> Handler:
        self._next_handler = handler
        return handler

    @abstractmethod
    def handle(self, request: Any) -> str:
        if self._next_handler:
            return self._next_handler.handle(request)

        return None


class AuthHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if not request.authorized:
            return f"{request.name} nie jest autoryzowany"
        else:
            return super().handle(request)


class FileHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request.is_file:
            return "Żądanie dotyczy pliku"
        else:
            return super().handle(request)


class RequestAmountHandler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if request.request_amount > 100:
            return "Przekroczono liczbę żądań"
        else:
            return super().handle(request)


class SqlInjectionHAndler(AbstractHandler):
    def handle(self, request: Any) -> str:
        if not request.is_sql_injection:
            return f"Żądanie użytkownika {request.name} zostało pomyślnie zwalidowane"
        else:
            return super().handle(request)


def client_code(handler: Handler) -> None:
    p1 = Req("Przemysław", True, True, 30, True)
    p2 = Req("Ryszard", False, True, 90, False)
    p3 = Req("Damian", True, False, 50, False)
    p4 = Req("Marcin", True, False, 110, False)
    p5 = Req("Tomek", True, False, 50, True)

    for user in [p1, p2, p3, p4, p5]:
        result = handler.handle(user)
        if result:
            print(f"  {result}")
        else:
            print(f"  Żądanie {user.name} zablokowane")


if __name__ == "__main__":
    # Zadanie 1
    main()
    
    # Zadanie 2
    context = Context(ConcreteStrategyA())
    context.do_some_business_logic()
    print()

    # Zadanie 3
    auth = AuthHandler()
    file = FileHandler()
    amount = RequestAmountHandler()
    sql = SqlInjectionHAndler()

    auth.set_next(file).set_next(amount).set_next(sql)

    client_code(auth)

